In [24]:
from pathlib import Path
import os

DATA_PATH = Path.cwd().parent / "data"

EOS_PATH = DATA_PATH / "EOS04 EDITED.xlsx"
SENTINEL_PATH = DATA_PATH / "SENTINEL Edited.xlsx"

os.path.isfile(EOS_PATH), os.path.isfile(SENTINEL_PATH)

(True, True)

In [25]:
import pandas as pd

eos = pd.ExcelFile(EOS_PATH)
sentinel = pd.ExcelFile(SENTINEL_PATH)

eos.sheet_names, sentinel.sheet_names

(['28-08-2022',
  '11-09-2022',
  '15-10-2022',
  '24-01-2023',
  '11-02-2023 ',
  '28-2-2023'],
 ['27-08-2022',
  '08-09-2022',
  '14-10-2022',
  '30-01-2023',
  '11-02-2023 ',
  '28-2-2023'])

# One Big Class For Experiments

In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
import warnings

warnings.filterwarnings('ignore', category=FutureWarning)


class ModelExperiments:
    def __init__(self, X_train, X_test, y_train, y_test, 
                 X_train_scaled=None, X_test_scaled=None, 
                 y_train_scaled=None, y_test_scaled=None):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.X_train_scaled = X_train_scaled
        self.X_test_scaled = X_test_scaled
        self.y_train_scaled = y_train_scaled
        self.y_test_scaled = y_test_scaled

    def fit_grid_search(self, model, param_grid, scaled=False, model_name="Model"):
        """
        Runs GridSearchCV for a given model + param_grid
        """
        print(f"\n=== Running {model_name} ===")
        
        X_train = self.X_train_scaled if scaled else self.X_train
        X_test = self.X_test_scaled if scaled else self.X_test
        y_train = self.y_train_scaled if scaled else self.y_train
        y_test = self.y_test_scaled if scaled else self.y_test

        grid_search = GridSearchCV(
            estimator=model,
            param_grid=param_grid,
            cv=3,
            n_jobs=-1,
            verbose=1
        )

        grid_search.fit(X_train, y_train)

        print("Best Parameters:", grid_search.best_params_)

        best_model = grid_search.best_estimator_
        print("Test R2 Score:", best_model.score(X_test, y_test))
        return best_model

    def run_all(self):
        """
        Runs all experiments: RF, XGB, AdaBoost, SVR
        """
        results = {}

        # Random Forest
        rf = RandomForestRegressor(random_state=10)
        rf_param_grid = {
            'n_estimators': [100, 200, 500],     
            'max_depth': [None, 5, 10, 20],      
            'min_samples_split': [2, 5, 10],     
            'min_samples_leaf': [1, 2, 4],       
            'max_features': ['sqrt', 'log2']     
        }
        results["RandomForest"] = self.fit_grid_search(rf, rf_param_grid, model_name="RandomForest")

        # XGBoost
        xgb = XGBRegressor(random_state=10, objective='reg:squarederror')
        xgb_param_grid = {
            'n_estimators': [100, 200, 500],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.05, 0.1],
            'subsample': [0.8, 1.0],
            'colsample_bytree': [0.8, 1.0]
        }
        results["XGBoost"] = self.fit_grid_search(xgb, xgb_param_grid, model_name="XGBoost")

        # AdaBoost
        ada = AdaBoostRegressor(
            estimator=DecisionTreeRegressor(random_state=10),
            random_state=10
        )
        ada_param_grid = {
            'n_estimators': [50, 100, 200],
            'learning_rate': [0.01, 0.05, 0.1, 1.0],
            'estimator__max_depth': [2, 3, 5, None],
            'estimator__min_samples_split': [2, 5, 10]
        }
        results["AdaBoost"] = self.fit_grid_search(ada, ada_param_grid, model_name="AdaBoost")

        # SVR (requires scaled data!)
        svr = SVR()
        svr_param_grid = {
            'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
            'C': [0.1, 1, 10, 100],
            'gamma': ['scale', 'auto', 0.01, 0.1, 1],
            'epsilon': [0.01, 0.1, 0.2, 0.5]
        }
        if self.X_train_scaled is not None:  # only run if scaled data provided
            results["SVR"] = self.fit_grid_search(svr, svr_param_grid, scaled=True, model_name="SVR")

        return results


# Only Sentinel Data

In [27]:
sentinel_dfs = []

for sheet_name in sentinel.sheet_names:
    df = pd.read_excel(SENTINEL_PATH, sheet_name=sheet_name)
    if 'Latitude (Centre of grid).1' in df.columns:
        df = df.drop(['Latitude (Centre of grid).1'], axis=1)
    df = df.rename(columns={'(θ)': 'angle'})
    sentinel_dfs.append(df)

sentinel_combined = pd.concat(sentinel_dfs, ignore_index=True)

sentinel_combined

,Sample Date & Time,Latitude (Centre of grid),Longitude (Centre of grid),VH-pol,VV-pol,angle,SM1 (%)
0,2022-08-08,22.526048,72.765011,-16.375600,-10.590500,40.048800,30.5
1,2022-08-08,22.525481,72.765028,-16.244300,-10.634800,41.349100,46.9
2,2022-08-08,22.525999,72.765663,-16.821400,-9.816820,42.283900,18.1
3,2022-08-08,22.527290,72.764707,-16.003700,-10.809500,42.692300,34.4
4,2022-08-08,22.527874,72.764718,-16.637400,-10.626300,43.860500,41.1
...,...,...,...,...,...,...,...
941,2023-02-28,22.523640,72.766727,-14.539132,-10.738241,41.797688,26.8
942,2023-02-28,22.523657,72.766016,-14.474763,-10.252501,41.254852,32.8
943,2023-02-28,22.524255,72.766024,-16.174562,-11.142242,42.046009,34.8
944,2023-02-28,22.524229,72.766601,-16.532534,-11.246845,41.706707,27.8


In [28]:
sentinel_combined['Day'] = sentinel_combined['Sample Date & Time'].dt.day
sentinel_combined['Month'] = sentinel_combined['Sample Date & Time'].dt.month
sentinel_combined['Year'] = sentinel_combined['Sample Date & Time'].dt.year

In [29]:
sentinel_combined = sentinel_combined.dropna()

## VH, VV, angle

In [30]:
X_cols = ['VH-pol', 'VV-pol', 'angle']
y_col = ['SM1 (%)']

X = sentinel_combined[X_cols]
y = sentinel_combined[y_col]

TRAIN_SIZE = 0.8

split_idx = int(len(X) * TRAIN_SIZE)

X_train, X_test = X.iloc[: split_idx], X.iloc[split_idx: ]
y_train, y_test = y.iloc[: split_idx], y.iloc[split_idx: ]

len(X_train), len(X_test)

(721, 181)

In [31]:
from sklearn.preprocessing import MinMaxScaler

mm = MinMaxScaler()

X_train_scaled = mm.fit_transform(X_train)
X_test_scaled = mm.transform(X_test)

y_train_scaled = mm.fit_transform(y_train)
y_test_scaled = mm.transform(y_test)

In [32]:
warnings.filterwarnings('always', category=FutureWarning)
sent_vv_vh_angle = ModelExperiments(X_train, X_test, y_train, y_test,
                    X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled).run_all()

sent_vv_vh_angle


=== Running RandomForest ===
Fitting 3 folds for each of 216 candidates, totalling 648 fits
Best Parameters: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 200}
Test R2 Score: -0.12419332552148332

=== Running XGBoost ===
Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 100, 'subsample': 1.0}
Test R2 Score: -0.10641217231750488

=== Running AdaBoost ===
Fitting 3 folds for each of 144 candidates, totalling 432 fits
Best Parameters: {'estimator__max_depth': 3, 'estimator__min_samples_split': 2, 'learning_rate': 0.1, 'n_estimators': 200}
Test R2 Score: -0.06356310889106287

=== Running SVR ===
Fitting 3 folds for each of 320 candidates, totalling 960 fits
Best Parameters: {'C': 10, 'epsilon': 0.1, 'gamma': 0.1, 'kernel': 'poly'}
Test R2 Score: -0.12875996535854006


{'RandomForest': RandomForestRegressor(max_depth=5, max_features='sqrt', min_samples_leaf=2,
                       min_samples_split=10, n_estimators=200, random_state=10),
 'XGBoost': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, ...),
 'AdaBoost': AdaBoostRegressor(estimator=DecisionTreeRegre

## VV, VH

In [33]:
X_cols = ['VH-pol', 'VV-pol']
y_col = ['SM1 (%)']

X = sentinel_combined[X_cols]
y = sentinel_combined[y_col]

TRAIN_SIZE = 0.8

split_idx = int(len(X) * TRAIN_SIZE)

X_train, X_test = X.iloc[: split_idx], X.iloc[split_idx: ]
y_train, y_test = y.iloc[: split_idx], y.iloc[split_idx: ]

len(X_train), len(X_test)

(721, 181)

In [34]:
from sklearn.preprocessing import MinMaxScaler

mm = MinMaxScaler()

X_train_scaled = mm.fit_transform(X_train)
X_test_scaled = mm.transform(X_test)

y_train_scaled = mm.fit_transform(y_train)
y_test_scaled = mm.transform(y_test)

In [35]:
warnings.filterwarnings('always', category=FutureWarning)
sent_vv_vh = ModelExperiments(X_train, X_test, y_train, y_test,
                    X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled).run_all()

sent_vv_vh


=== Running RandomForest ===
Fitting 3 folds for each of 216 candidates, totalling 648 fits
Best Parameters: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
Test R2 Score: -0.13711664977999827

=== Running XGBoost ===
Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Test R2 Score: -0.07659471035003662

=== Running AdaBoost ===
Fitting 3 folds for each of 144 candidates, totalling 432 fits
Best Parameters: {'estimator__max_depth': 3, 'estimator__min_samples_split': 10, 'learning_rate': 1.0, 'n_estimators': 50}
Test R2 Score: -0.15220357149815533

=== Running SVR ===
Fitting 3 folds for each of 320 candidates, totalling 960 fits
Best Parameters: {'C': 1, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'rbf'}
Test R2 Score: -0.31303685667649916


{'RandomForest': RandomForestRegressor(max_depth=5, max_features='sqrt', min_samples_leaf=4,
                       min_samples_split=10, random_state=10),
 'XGBoost': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, ...),
 'AdaBoost': AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=3,


## VV, VH, angle, day, month, year

In [36]:
X_cols = ['VH-pol', 'VV-pol', 'angle', 'Day', 'Month', 'Year']
y_col = ['SM1 (%)']

X = sentinel_combined[X_cols]
y = sentinel_combined[y_col]

TRAIN_SIZE = 0.8

split_idx = int(len(X) * TRAIN_SIZE)

X_train, X_test = X.iloc[: split_idx], X.iloc[split_idx: ]
y_train, y_test = y.iloc[: split_idx], y.iloc[split_idx: ]

len(X_train), len(X_test)

(721, 181)

In [37]:
from sklearn.preprocessing import MinMaxScaler

mm = MinMaxScaler()

X_train_scaled = mm.fit_transform(X_train)
X_test_scaled = mm.transform(X_test)

y_train_scaled = mm.fit_transform(y_train)
y_test_scaled = mm.transform(y_test)

In [38]:
warnings.filterwarnings('always', category=FutureWarning)
sent_vv_vh_angle_dmy = ModelExperiments(X_train, X_test, y_train, y_test,
                    X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled).run_all()

sent_vv_vh_angle_dmy


=== Running RandomForest ===
Fitting 3 folds for each of 216 candidates, totalling 648 fits
Best Parameters: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 500}
Test R2 Score: -0.18304661755876372

=== Running XGBoost ===
Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Test R2 Score: -0.09249842166900635

=== Running AdaBoost ===
Fitting 3 folds for each of 144 candidates, totalling 432 fits
Best Parameters: {'estimator__max_depth': 5, 'estimator__min_samples_split': 2, 'learning_rate': 1.0, 'n_estimators': 100}
Test R2 Score: -0.3238648811035951

=== Running SVR ===
Fitting 3 folds for each of 320 candidates, totalling 960 fits
Best Parameters: {'C': 1, 'epsilon': 0.1, 'gamma': 0.1, 'kernel': 'sigmoid'}
Test R2 Score: -0.19253638204632217


{'RandomForest': RandomForestRegressor(max_depth=5, max_features='sqrt', min_samples_split=5,
                       n_estimators=500, random_state=10),
 'XGBoost': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.01, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, ...),
 'AdaBoost': AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=5, ran

# Only EOS Data

In [39]:
eos_dfs = []

for sheet_name in eos.sheet_names:
    df = pd.read_excel(EOS_PATH, sheet_name=sheet_name)
    eos_dfs.append(df)

eos_combined = pd.concat(eos_dfs, ignore_index=True)

eos_combined

,Sample Date & Time,Latitude (Centre of grid),Longitude (Centre of grid),HH-pol,HV-pol,SM1 (%)
0,2022-08-28,22.526048,72.765011,-4.99884,-13.33651,30.5
1,2022-08-28,22.525481,72.765028,-8.76236,-16.08855,46.9
2,2022-08-28,22.525999,72.765663,-7.11428,-11.90641,18.1
3,2022-08-28,22.527290,72.764707,-8.32358,-15.11733,34.4
4,2022-08-28,22.527874,72.764718,-5.27314,-15.93518,41.1
...,...,...,...,...,...,...
892,2023-02-28,22.523640,72.766727,-8.69274,-14.15688,26.8
893,2023-02-28,22.523657,72.766016,-6.48271,-15.56076,32.8
894,2023-02-28,22.524255,72.766024,-6.20660,-14.73141,34.8
895,2023-02-28,22.524229,72.766601,-9.43414,-20.52987,27.8


In [40]:
eos_combined['Month'] = eos_combined['Sample Date & Time'].dt.month
eos_combined['Day'] = eos_combined['Sample Date & Time'].dt.day
eos_combined['Year'] = eos_combined['Sample Date & Time'].dt.year

## HH, HV

In [41]:
X_cols = ['HH-pol', 'HV-pol']
y_col = ['SM1 (%)']

TRAIN_SIZE = 0.8

X = eos_combined[X_cols]
y = eos_combined[y_col]

split_idx = int(len(X) * TRAIN_SIZE)

X_train, X_test = X.iloc[: split_idx], X.iloc[split_idx: ]
y_train, y_test = y.iloc[: split_idx], y.iloc[split_idx: ]

len(X_train), len(X_test)

(717, 180)

In [42]:
from sklearn.preprocessing import MinMaxScaler

X_train_scaled = mm.fit_transform(X_train)
X_test_scaled = mm.transform(X_test)

y_train_scaled = mm.fit_transform(y_train)
y_test_scaled = mm.transform(y_test)

In [44]:
eos_hh_hv = ModelExperiments(X_train, X_test, y_train, y_test,
                             X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled).run_all()


=== Running RandomForest ===
Fitting 3 folds for each of 216 candidates, totalling 648 fits
Best Parameters: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
Test R2 Score: 0.007397434859398366

=== Running XGBoost ===
Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
Test R2 Score: -0.2304222583770752

=== Running AdaBoost ===
Fitting 3 folds for each of 144 candidates, totalling 432 fits
Best Parameters: {'estimator__max_depth': 2, 'estimator__min_samples_split': 2, 'learning_rate': 1.0, 'n_estimators': 50}
Test R2 Score: 0.024421436193262203

=== Running SVR ===
Fitting 3 folds for each of 320 candidates, totalling 960 fits
Best Parameters: {'C': 1, 'epsilon': 0.1, 'gamma': 'scale', 'kernel': 'sigmoid'}
Test R2 Score: -0.18142027044210884


## HH, HV, Day, Month, Year

In [45]:
X_cols = ['HH-pol', 'HV-pol', 'Day', 'Month', 'Year']
y_col = ['SM1 (%)']

TRAIN_SIZE = 0.8

X = eos_combined[X_cols]
y = eos_combined[y_col]

split_idx = int(len(X) * TRAIN_SIZE)

X_train, X_test = X.iloc[: split_idx], X.iloc[split_idx: ]
y_train, y_test = y.iloc[: split_idx], y.iloc[split_idx: ]

len(X_train), len(X_test)

(717, 180)

In [46]:
from sklearn.preprocessing import MinMaxScaler

X_train_scaled = mm.fit_transform(X_train)
X_test_scaled = mm.transform(X_test)

y_train_scaled = mm.fit_transform(y_train)
y_test_scaled = mm.transform(y_test)

In [47]:
eos_hh_hv_dmy = ModelExperiments(X_train, X_test, y_train, y_test,
                             X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled).run_all()


=== Running RandomForest ===
Fitting 3 folds for each of 216 candidates, totalling 648 fits
Best Parameters: {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Test R2 Score: -0.01605000034572779

=== Running XGBoost ===
Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best Parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Test R2 Score: -0.1027759313583374

=== Running AdaBoost ===
Fitting 3 folds for each of 144 candidates, totalling 432 fits
Best Parameters: {'estimator__max_depth': 2, 'estimator__min_samples_split': 2, 'learning_rate': 1.0, 'n_estimators': 50}
Test R2 Score: -0.10198427533640286

=== Running SVR ===
Fitting 3 folds for each of 320 candidates, totalling 960 fits
Best Parameters: {'C': 100, 'epsilon': 0.01, 'gamma': 'scale', 'kernel': 'linear'}
Test R2 Score: -1.231915905531031


# EOS and Sentinel Data Combined

In [48]:
eos.sheet_names, sentinel.sheet_names

(['28-08-2022',
  '11-09-2022',
  '15-10-2022',
  '24-01-2023',
  '11-02-2023 ',
  '28-2-2023'],
 ['27-08-2022',
  '08-09-2022',
  '14-10-2022',
  '30-01-2023',
  '11-02-2023 ',
  '28-2-2023'])